<a href="https://colab.research.google.com/github/lindonjonhson/irish-songs-generator/blob/main/william_bot_yeats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import numpy as np

In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/learning-datasets/irish-lyrics-eof.txt \
    -O /tmp/irish-lyrics-eof.txt

--2024-04-08 17:08:56--  https://storage.googleapis.com/learning-datasets/irish-lyrics-eof.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.198.207, 173.194.206.207, 173.194.74.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.198.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68970 (67K) [text/plain]
Saving to: ‘/tmp/irish-lyrics-eof.txt’

/tmp/irish-lyrics-e 100%[===================>]  67.35K  --.-KB/s    in 0.002s  

2024-04-08 17:08:56 (38.1 MB/s) - ‘/tmp/irish-lyrics-eof.txt’ saved [68970/68970]



In [ ]:
tokenizer = Tokenizer()

data = open('/tmp/irish-lyrics-eof.txt').read()

corpus = data.lower().split("\n")

#Fitting the tokenizer with the sentences
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1   #2690

When generating text, we do not need a test set

In [ ]:
# Creating sequences
input_sequences =[]

for line in corpus:
  token_list = tokenizer.texts_to_sequences([line])[0]  # Transform the lines in arrays with the codes from the word index
  #print(token_list)
  #print(line)
  #print(len(token_list))
  for i in range(1, len(token_list)): # Now we will work with ngrams, which are the "parts" of a line until it is completemax
    #print(token_list[:i+1])
    n_gram_sequence = token_list[:i+1]
    #print(n_gram_sequence)
    input_sequences.append(n_gram_sequence)

In [ ]:
# Retrieving the length of the longest of the sentences
max_sequence_len = max(len(x) for x in input_sequences)

#Padding
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In [ ]:
# Generating Xs and Ys/labels by splitting the arrays

xs = input_sequences[:, :-1]
labels = input_sequences[:, -1]

In [ ]:
# Now we will make the labels categorical and hot-encoded
ys = tf.keras.utils.to_categorical(labels, num_classes=total_words)

In [ ]:
# Train the model with the data
model = Sequential()
model.add(Embedding(total_words, 240, input_length=max_sequence_len))
model.add(Bidirectional(LSTM(150)))
model.add(Dense(total_words, activation='softmax'))
adam = Adam(learning_rate=0.01)
model.compile(loss='categorical_entropy', optimizer=adam, metrics=['accuracy'])
# history = model.fit(xs, ys, epochs=100, verbose=1)
history = model.fit(xs, ys, epochs=100, verbose=1)

Epoch 1/100


ValueError: in user code:

    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py", line 1150, in train_step
        y_pred = self(x, training=True)
    File "/usr/local/lib/python3.10/dist-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/usr/local/lib/python3.10/dist-packages/keras/src/engine/input_spec.py", line 298, in assert_input_compatibility
        raise ValueError(

    ValueError: In

In [ ]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(150)))
model.add(Dense(total_words, activation='softmax'))
adam = Adam(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
#earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=0, mode='auto')
history = model.fit(xs, ys, epochs=100, verbose=1)
#print model.summary()
print(model)


Epoch 1/100
377/377 [==============================] - 40s 92ms/step - loss: 6.7516 - accuracy: 0.0652
Epoch 2/100
377/377 [==============================] - 35s 94ms/step - loss: 6.2532 - accuracy: 0.0752
Epoch 3/100
377/377 [==============================] - 35s 93ms/step - loss: 5.9931 - accuracy: 0.0836
Epoch 4/100
377/377 [==============================] - 36s 96ms/step - loss: 5.6918 - accuracy: 0.1012
Epoch 5/100
377/377 [==============================] - 34s 89ms/step - loss: 5.3571 - accuracy: 0.1149
Epoch 6/100
377/377 [==============================] - 34s 90ms/step - loss: 4.9929 - accuracy: 0.1327
Epoch 7/100
377/377 [==============================] - 33s 87ms/step - loss: 4.6367 - accuracy: 0.1527
Epoch 8/100
377/377 [==============================] - 34s 91ms/step - loss: 4.2816 - accuracy: 0.1798
Epoch 9/100
377/377 [==============================] - 34s 91ms/step - loss: 3.9316 - accuracy: 0.2172
Epoch 10/100
377/377 [==============================] - 33s 86ms/step - l

In [ ]:
import matplotlib.pyplot as plt


def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.show()

In [ ]:
plot_graphs(history, 'accuracy')


In [ ]:
seed_text = "In the darkest night"
next_words = 100

for _ in range(next_words):
	token_list = tokenizer.texts_to_sequences([seed_text])[0]
	token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
	predicted = np.argmax(model.predict(token_list), axis=-1)
	output_word = ""
	for word, index in tokenizer.word_index.items():
		if index == predicted:
			output_word = word
			break
	seed_text += " " + output_word
print(seed_text)